<a href="https://colab.research.google.com/github/sjung-stat/Customer-Support-Chat-Intent-Classification/blob/main/Model%20Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before building a text classification model using BERT, we need to preprocess the dataset. First of all, we do one-hot encoding on our target variable. And after that, we use one of the companions of BERT models which is designed to work alongside BERT models, and its purpose is to transform unprocessed textual inputs into the appropriate input format required by BERT. More details can be found below.

In [6]:
# Import necessary libraries

from google.colab import drive
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [2]:
# Load the preprocessed training and testing data
drive.mount('/content/drive')

with open('/content/drive/MyDrive/df_training_complete.pkl', 'rb') as f:   # Load `df_training_complete` from Google Drive
    df_training_complete = pickle.load(f)

with open('/content/drive/MyDrive/df_testing_copy.pkl', 'rb') as f:   # Load `df_testing_copy` from Google Drive
    df_testing_complete = pickle.load(f)

Mounted at /content/drive


# Preprocessing

## One Hot Encoding

In [7]:
# Split training and validation sets and reformat testing set. 
trainfeatures, validfeatures, trainlabels, validlabels = train_test_split(df_training_complete['text'],df_training_complete['category'], stratify=df_training_complete['category'], test_size=0.2)

testfeatures=df_testing_complete.copy()
testlabels=testfeatures.pop("category")


# One-Hot-Encoding of class-labels
binarizer=LabelBinarizer()  

trainlabels=binarizer.fit_transform(trainlabels.values)
validlabels=binarizer.transform(validlabels.values)
testlabels=binarizer.transform(testlabels.values)
trainfeatures = pd.DataFrame(trainfeatures)
validfeatures = pd.DataFrame(validfeatures)

In [9]:
print("Total number of training examples: ", trainfeatures.shape[0])
print("Total number of validation examples: ", validfeatures.shape[0])
print("Total number of testing examples: ", testfeatures.shape[0])

Total number of training examples:  12733
Total number of validation examples:  3184
Total number of testing examples:  3080


In [10]:
!pip install -q tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 23.8 MB/s eta 0:00:00


In [11]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.preprocessing import LabelBinarizer

In [12]:
# Note that the following preprocessing model cannot take pandas dataframe as input
# Need to convert the input data (text) into list
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") # The text data we will be working with will undergo preprocessing using a TensorFlow model.
                                                                                              # As this preprocessor is a TensorFlow model, it can be easily integrated directly into your own model.

13

# Model Building

For this intent classification project, we use BERT which is a pretrained language model. BERT is useful for intent classification because it is pre-trained on a large corpus of text data, allowing it to understand the nuances of natural language. Additionally, BERT uses a bidirectional approach, which means it can analyze a text input in both directions, allowing it to better understand the context and meaning of the input. By using BERT as a pre-processing step, you can improve the accuracy and efficiency of your intent classification model.

In [15]:
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1")

In [17]:
def intent_classification_bert():
  
  # Initializing the BERT layers
  input_text = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text') # define an input tensor --> the input data to the model will be a string of variable length
  text_preprocessed = bert_preprocess(input_text) # This layer converts the input string into a format that can be understood by the BERT model
  output_bert = bert_model(text_preprocessed) # This layer encodes the input text using the BERT model and returns the encoded outputs
                                              # After that, this will be fed into the neural network layers.

  # Initializing the neural network layers
  encoded_text = output_bert['pooled_output']
  layer_1 = tf.keras.layers.Dense(512, activation='relu')(encoded_text)
  layer_2 = tf.keras.layers.Dense(256, activation='relu')(layer_1)
  layer_3 = tf.keras.layers.Dense(128, activation='relu')(layer_2)
  layer_4 = tf.keras.layers.Dropout(0.1)(layer_3) # This layer will be used to prevent model overfitting
                                                                                 # We will use 0.1% of the neurons to handle overfitting
  output = tf.keras.layers.Dense(trainlabels.shape[1], activation='softmax')(layer_4)  # It only has one neuron. We also initialize the activation function as sigmoid. 
                                                                                 # sigmoid is used when we have output values that between 0 and 1. 
                                                                                 # In our case, when making predictions, 
                                                                                 # the prediction probability will lie between 0 and 1. That’s why it is best suited.
                                                                                 # We also name the layer as output because this is our output layer.
  return tf.keras.Model(input_text, output)

In [18]:
classifier_model = intent_classification_bert()

loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True) # Since this is a non-binary classification problem and the model outputs probabilities, 
                                                                 # you’ll use losses.CategoricalCrossentropy loss function.
metrics = tf.metrics.CategoricalAccuracy()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [19]:
epochs=20
optimizer=tf.keras.optimizers.Adam(1e-5)
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [20]:
history = classifier_model.fit(x=trainfeatures, y=trainlabels,
                               validation_data=(validfeatures,validlabels),
                               batch_size=32,
                               #class_weight=class_weights_dict,
                               epochs=epochs)

Epoch 1/20


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


398/398 [==============================] - ETA: 0s - loss: 2.5312 - categorical_accuracy: 0.1350

/usr/local/lib/python3.9/dist-packages/keras/backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


398/398 [==============================] - 99s 219ms/step - loss: 2.5312 - categorical_accuracy: 0.1350 - val_loss: 2.4293 - val_categorical_accuracy: 0.2220
Epoch 2/20
398/398 [==============================] - 87s 218ms/step - loss: 2.3493 - categorical_accuracy: 0.2544 - val_loss: 2.2450 - val_categorical_accuracy: 0.3194
Epoch 3/20
398/398 [==============================] - 86s 217ms/step - loss: 2.1798 - categorical_accuracy: 0.3273 - val_loss: 2.0829 - val_categorical_accuracy: 0.3728
Epoch 4/20
398/398 [==============================] - 82s 206ms/step - loss: 2.0371 - categorical_accuracy: 0.3653 - val_loss: 1.9543 - val_categorical_accuracy: 0.4130
Epoch 5/20
398/398 [==============================] - 87s 218ms/step - loss: 1.9231 - categorical_accuracy: 0.4022 - val_loss: 1.8528 - val_categorical_accuracy: 0.4400
Epoch 6/20
398/398 [==============================] - 86s 217ms/step - loss: 1.8315 - categorical_accuracy: 0.4286 - val_loss: 1.7696 - val_categorical_accuracy: 0.46